In [ ]:
import json
import os
import pickle
import re
from pathlib import Path

import pandas as pd
import spacy
from dotenv import load_dotenv
from spacy.language import Language
from spacy.tokens import Doc

from discharge_summaries.openai_llm.chat_models import AzureOpenAIChatModel
from discharge_summaries.openai_llm.message import Message, Role
from discharge_summaries.schemas.mimic import BHC
from discharge_summaries.snomed.lookup import SnomedLookup
from discharge_summaries.snomed.retriever import SnomedRetriever

In [ ]:
load_dotenv()

In [ ]:
MIMIC_III_DIR = (
    Path.cwd().parent / "data" / "physionet.org" / "files" / "mimiciii" / "1.4"
)
BHC_FPATH = MIMIC_III_DIR / "BHCS.json"
PHYSICIAN_NOTE_FPATH = MIMIC_III_DIR / "physician_notes_mimic.csv"

SNOMED_DIR = Path.cwd().parent / "data" / "snomed"
TUNED_PHRASE_MATCHER_FPATH = SNOMED_DIR / "tuned_snomed_phrase_matcher.pkl"
PROMPT_MESSAGE_FPATH = Path.cwd() / "prompt_message.txt"
AZURE_ENGINE = "gpt-35-turbo"
AZURE_API_VERSION = "2023-07-01-preview"
EXAMPLE_DIR = Path.cwd() / "example"

In [ ]:
snomed_phrase_matcher = pickle.load(TUNED_PHRASE_MATCHER_FPATH.open("rb"))

In [ ]:
snomed_lookup = SnomedLookup.load(SNOMED_DIR)

In [ ]:
llm = AzureOpenAIChatModel(
    api_base=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    api_version=AZURE_API_VERSION,
    engine=AZURE_ENGINE,
    temperature=0,
    timeout=20,
)

In [ ]:
@Language.component("mimic_section_splitter")
def regex_sentence_splitter(doc: Doc) -> Doc:
    matches = re.finditer(r"(?<=\n).*?[^\n](?=:)|(?<=\n{2}).", doc.text)
    sentence_starts = {match.start() for match in matches}
    for token in doc:
        token.is_sent_start = True if token.idx in sentence_starts else False

    return doc


nlp_with_section_splitter = spacy.load("en_core_sci_lg", disable=["parser", "ner"])
nlp_with_section_splitter.add_pipe("lower_case_lemmas", last=True)
nlp_with_section_splitter.add_pipe("mimic_section_splitter")

In [ ]:
snomed_retriever = SnomedRetriever(
    snomed_phrase_matcher, snomed_lookup, nlp_with_section_splitter
)

In [ ]:
bhcs = [BHC(**bhc_dict) for bhc_dict in json.loads(BHC_FPATH.read_text())]
physician_notes = pd.read_csv(PHYSICIAN_NOTE_FPATH)

In [ ]:
for idx, bhc in enumerate(bhcs[:100]):
    print(idx)
    print(bhc.assessment_and_plan)
    print("*" * 80)

In [ ]:
sample_bhc = bhcs[10]
sample_physician_notes = physician_notes[
    physician_notes["HADM_ID"] == sample_bhc.hadm_id
].sort_values("CHARTTIME")

In [ ]:
(EXAMPLE_DIR / "bhc.txt").write_text(sample_bhc.full_text)
with (EXAMPLE_DIR / "bhc.txt").open("a") as f:
    f.write(f"\n{'*'*80}\n{json.dumps(sample_bhc.dict(), indent=4)}")

## First Paragraph

Bit long I think... Check average

In [ ]:
examples = "\n\n".join(bhc.assessment_and_plan for bhc in bhcs[:3])
first_para_system_message = Message(
    role=Role.SYSTEM,
    content=f"""You are a consultant doctor completing a medical discharge summary.
Your task is to write the first paragraph of the summary.
The paragraph should be 30 words long.
The paragraph must include the patient's:
- Age
- Gender
- Past medical history
- Reason for hospital admission
This information can be found in the admission note provided by the user.

The following are examples of first paragraphs:
{examples}""",
)

In [ ]:
first_para_user_message = Message(
    role=Role.USER,
    content=f"""Admission Note 

{sample_physician_notes.iloc[0]["TEXT"]}
Please write the first paragraph of the discharge summary using the admission note and the requirements given in the system message.
""",  # noqa
)

In [ ]:
first_para_response = llm.query([first_para_system_message, first_para_user_message])

In [ ]:
message_delimiter = "\n" + ("*" * 80) + "\n"
message_strings = message_delimiter.join(
    f"{message.role}\n{message.content}"
    for message in [
        first_para_system_message,
        first_para_user_message,
        first_para_response,
    ]
)
message_strings += f"\n{message_delimiter}\nMIMIC BHC\n{sample_bhc.assessment_and_plan}"
(EXAMPLE_DIR / "first_para.txt").write_text(message_strings)

## Findings

In [ ]:
examples = "\n\n".join(
    "\n".join(
        para.heading
        for para in bhc.problem_sections
        if snomed_phrase_matcher(para.heading)
    )
    for bhc in bhcs[:3]
)
findings_system_message = Message(
    role=Role.SYSTEM,
    content=f"""You are a consultant doctor completing a medical discharge summary.
Your task is to list the main clinical findings made during the patient's stay.
Each finding should be on a new line.
Use Snomed CT preferred terms.
This information can be found in the physician note provided by the user.

The following are examples of previous patient's clinical findings:
{examples}""",
)

In [ ]:
findings_user_message = Message(
    role=Role.USER,
    content=f"""Physician Note

{sample_physician_notes.iloc[-1]["TEXT"]}
Please write the list findings using the physician note and the requirements given in the system message.
""",
)

In [ ]:
findings_response = llm.query([findings_system_message, findings_user_message])
gpt_findings = set(findings_response.content.split("\n"))

In [ ]:
message_delimiter = "\n" + ("*" * 80) + "\n"
output_strings = [
    f"{message.role}\n{message.content}"
    for message in [findings_system_message, findings_user_message]
]
gpt_heading_strings = "\n".join(sorted(gpt_findings))
output_strings.append(f"{Role.ASSISTANT}\n{gpt_heading_strings}")
heading_strings = "\n".join(
    sorted(para.heading for para in sample_bhc.problem_sections)
)
output_strings.append(f"Mimic Headings\n{heading_strings}")
(EXAMPLE_DIR / "findings.txt").write_text(message_delimiter.join(output_strings))

## Retrieval

In [ ]:
gpt_findings

In [ ]:
retrieved_findings = snomed_retriever(sample_physician_notes["TEXT"], gpt_findings)

In [ ]:
timestamps = sample_physician_notes["CHARTTIME"].tolist()
finding_to_extract = {
    finding: "\n\n".join(
        f"Physician Note {note_idx}, {timestamps[note_idx]}\n{sent.text.strip()}"
        for note_idx, sent in note_idx_and_sents
    )
    for finding, note_idx_and_sents in retrieved_findings.items()
}

In [ ]:
finding_to_extract_strs = [
    f"{finding}\n{extract}" for finding, extract in finding_to_extract.items()
]
(EXAMPLE_DIR / "retrieval.txt").write_text(
    message_delimiter.join(finding_to_extract_strs)
)